In [19]:
import json
import ast
from itertools import chain
from sklearn.metrics import classification_report
# from sklearn_crfsuite.metrics import flatten
# from sklearn.metrics import classification_report

from datasets import load_from_disk

from reference_parsing.config import (LABEL2ID)

In [2]:
prepared_dataset = load_from_disk("./datasets/prepared_dataset")

Loading dataset from disk:   0%|          | 0/19 [00:00<?, ?it/s]

In [8]:
subset = prepared_dataset["test"].select(range(100))
subset[0]

# subset.to_csv("train_first100.csv", index=False)

{'tokens': ['Geiger',
  ',',
  'R',
  '.',
  '&',
  '',
  'Heller',
  ',',
  'D',
  '.',
  '.',
  '2012',
  '.',
  'Financial',
  'trends',
  'in',
  'higher',
  'education',
  ':',
  'The',
  'United',
  'States',
  '.',
  'Voprosy',
  'Obrazovaniya',
  '/',
  'Educational',
  'Studies',
  '.',
  'Moscow',
  ',',
  '(',
  '3',
  ')',
  ':',
  '5–29',
  '.',
  'http://dx.doi.org/10.17323/1814-9545-2012-3-5-29',
  '.'],
 'labels': ['B-AUTHOR',
  'B-PUNC',
  'I-AUTHOR',
  'B-PUNC',
  'B-PUNC',
  'I-AUTHOR',
  'I-AUTHOR',
  'B-PUNC',
  'I-AUTHOR',
  'B-PUNC',
  'B-PUNC',
  'B-YEAR',
  'B-PUNC',
  'B-TITLE',
  'I-TITLE',
  'I-TITLE',
  'I-TITLE',
  'I-TITLE',
  'B-PUNC',
  'I-TITLE',
  'I-TITLE',
  'I-TITLE',
  'B-PUNC',
  'B-CONTAINER-TITLE',
  'I-CONTAINER-TITLE',
  'B-PUNC',
  'I-CONTAINER-TITLE',
  'I-CONTAINER-TITLE',
  'B-PUNC',
  'I-CONTAINER-TITLE',
  'B-PUNC',
  'B-PUNC',
  'B-ISSUE',
  'B-PUNC',
  'B-PUNC',
  'B-PAGE',
  'B-PUNC',
  'B-URL',
  'B-PUNC']}

In [20]:
label2id = LABEL2ID
id2label = {v: k for k, v in label2id.items()}
label_order = list(label2id.keys())

In [15]:
with open("train_tokens.json", "w", encoding="utf-8") as f_toks, \
     open("train_labels.json", "w", encoding="utf-8") as f_labs:
    for ex in subset:
        # dumps will produce e.g. ["Chen","K-S",".", ...]
        f_toks.write(json.dumps(ex["tokens"], ensure_ascii=False) + "\n")
        f_labs.write(json.dumps(ex["labels"],  ensure_ascii=False) + "\n")

## Comparing results

In [162]:
import re

d = r'''
{
    "Girgis": "B-AUTHOR",
    ",": "B-PUNC",
    "Safwat": "I-AUTHOR",
    ",": "B-PUNC",
    "Ibrahim": "I-AUTHOR",
    "Ramzy": "I-AUTHOR",
    ",": "B-PUNC",
    "Susan": "I-AUTHOR",
    "C": "I-AUTHOR",
    ".": "B-PUNC",
    "Baer": "I-AUTHOR",
    ",": "B-PUNC",
    "and": "O",
    "Mary": "I-AUTHOR",
    "R": "I-AUTHOR",
    ".": "B-PUNC",
    "Schwartz": "I-AUTHOR",
    ",": "B-PUNC",
    "“": "B-PUNC",
    "Fine": "B-TITLE",
    "Needle": "I-TITLE",
    "Aspiration": "I-TITLE",
    "Diagnosis": "I-TITLE",
    "of": "I-TITLE",
    "Transitional": "I-TITLE",
    "Cell": "I-TITLE",
    "Carcinoma": "I-TITLE",
    "Metastatic": "I-TITLE",
    "to": "I-TITLE",
    "the": "I-TITLE",
    "Brain": "I-TITLE",
    ",”": "B-PUNC",
    "Acta": "B-CONTAINER-TITLE",
    "Cytologica": "I-CONTAINER-TITLE",
    ",": "B-PUNC",
    "xliii": "B-VOLUME",
    "/": "B-PUNC",
    "2": "B-ISSUE",
    "(": "B-PUNC",
    "1999": "B-YEAR",
    ")": "B-PUNC",
    ",": "B-PUNC",
    "pp": "B-PAGE",
    ".": "B-PUNC",
    "235–238": "I-PAGE",
    ".": "B-PUNC"
}
'''

values = re.findall(r'"\s*:\s*"([^"]+)"', d)
values = ['B-PUNC' if x==': ' else x for x in values]
print(values)
# with open("predectied_labels.txt", "a", encoding="utf-8") as f:
#     f.write(str(values))          # writes "['1989', '.', 'Intellectual', ...]"
#     f.write("\n")

['B-AUTHOR', 'B-PUNC', 'I-AUTHOR', 'B-PUNC', 'I-AUTHOR', 'I-AUTHOR', 'B-PUNC', 'I-AUTHOR', 'I-AUTHOR', 'B-PUNC', 'I-AUTHOR', 'B-PUNC', 'O', 'I-AUTHOR', 'I-AUTHOR', 'B-PUNC', 'I-AUTHOR', 'B-PUNC', 'B-PUNC', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'B-PUNC', 'B-CONTAINER-TITLE', 'I-CONTAINER-TITLE', 'B-PUNC', 'B-VOLUME', 'B-PUNC', 'B-ISSUE', 'B-PUNC', 'B-YEAR', 'B-PUNC', 'B-PUNC', 'B-PAGE', 'B-PUNC', 'I-PAGE', 'B-PUNC']


In [167]:
true_path = "train_labels.txt"
pred_path = "predectied_labels.txt"

with open(true_path, "r", encoding="utf-8") as f:
    true_seqs = [ast.literal_eval(line) for line in f if line.strip()]

with open(pred_path, "r", encoding="utf-8") as f:
    pred_seqs = [ast.literal_eval(line) for line in f if line.strip()]

In [168]:
for i,(t_seq, p_seq) in enumerate(zip(true_seqs, pred_seqs)):
    if len(t_seq) != len(p_seq):
        print(f"  mismatch at index {i}:  true len={len(t_seq)}, pred len={len(p_seq)}")

In [169]:
y_true = list(chain.from_iterable(true_seqs))
y_pred = list(chain.from_iterable(pred_seqs))

In [170]:
print(classification_report(
    y_true,
    y_pred,
    labels=label_order,
    zero_division=0
))

                   precision    recall  f1-score   support

         B-AUTHOR       0.44      0.99      0.61        80
         I-AUTHOR       0.95      0.67      0.79       375
           B-YEAR       0.85      0.99      0.91        87
           I-YEAR       0.00      0.00      0.00         0
          B-TITLE       0.91      0.94      0.92        95
          I-TITLE       0.97      0.93      0.95       674
B-CONTAINER-TITLE       0.95      0.90      0.92        96
I-CONTAINER-TITLE       0.97      0.88      0.93       250
         B-VOLUME       0.90      0.97      0.94        68
         I-VOLUME       0.00      0.00      0.00         0
          B-ISSUE       0.77      0.85      0.81        27
          I-ISSUE       0.00      0.00      0.00         0
           B-PAGE       0.86      0.90      0.88        79
           I-PAGE       0.00      0.00      0.00         0
           B-ISBN       0.00      0.00      0.00         0
           I-ISBN       0.00      0.00      0.00       